In [34]:
import os
import sys
import time
from datetime import datetime as dt
import numpy as np
from matplotlib import pyplot as plt, ticker
from pysar.simulation.plot import *
from pysar.objects import timeseries, ifgramStack, sensor, gps
from pysar.utils import readfile, ptime, plot as pp, writefile, utils as ut

work_dir = '/Users/yunjunz/development/python/2018_ComputGeosci_PySAR/pysar_vs_gps'
os.chdir(work_dir)
print('Go to directory: '+work_dir)

site = 'GV02'

gps_dir = '/Users/yunjunz/insarlab/Galapagos/GPS'
pysar_dir = '/Users/yunjunz/insarlab/Galapagos/GalapagosSenDT128/PYSAR/GEOCODE'
ts_file = os.path.join(pysar_dir, 'geo_timeseries_ECMWF_demErr_refDate_plane.h5')

Go to directory: /Users/yunjunz/development/python/2018_ComputGeosci_PySAR/pysar_vs_gps


In [35]:
## Read GPS
gps_obj = gps(site=site, data_dir=gps_dir)
gps_obj.open()

gps_dis = ut.enu2los(gps_obj.dis_e, gps_obj.dis_n, gps_obj.dis_u)
gps_times = gps_obj.times

## Read InSAR
atr = readfile.read_attribute(ts_file)
y = ut.coord_geo2radar(gps_obj.site_lat, atr, 'lat')
x = ut.coord_geo2radar(gps_obj.site_lon, atr, 'lon')
box = (x, y, x+1, y+1)

ts_obj = timeseries(ts_file)
ts_obj.open()
pysar_dis = ts_obj.read(box=box)
pysar_times = ts_obj.times

downloading GV02 from http://geodesy.unr.edu/gps_timeseries/tenv3/IGS08/GV02.IGS08.tenv3
calculating station lat/lon
reading time and displacement in east/north/vertical direction
open timeseries file: geo_timeseries_ECMWF_demErr_refDate_plane.h5
reading timeseries data from file: /Users/yunjunz/insarlab/Galapagos/GalapagosSenDT128/PYSAR/GEOCODE/geo_timeseries_ECMWF_demErr_refDate_plane.h5 ...


In [39]:
## Plot
title = 'Station ID = {}, lat={:.4f}, lon={:.4f}'.format(site, gps_obj.site_lat, gps_obj.site_lon)
comm_idx = 0
while comm_idx < len(pysar_times):
    if pysar_times[comm_idx] not in gps_times:
        comm_idx += 1
    else:
        break
if comm_idx == len(pysar_times):
    raise RuntimeError('InSAR and GPS do not share ANY date.')

comm_date = pysar_times[comm_idx]
print('Common date: {}'.format(comm_date))
pysar_dis -= pysar_dis[comm_idx]
gps_dis -= gps_dis[gps_times.index(comm_date)]

plt.figure(figsize=(10, 5))
plt.scatter(gps_times, gps_dis, s=0.2**2, label='GPS')
plt.scatter(pysar_times, pysar_dis, s=2**2, label='InSAR')
plt.xlabel('Time (year)')
plt.ylabel('LOS displacement (m)')
plt.title(title)

out_file = '{}.jpg'.format(site)
plt.savefig(out_file, bbox_inches='tight', transparent=True, dpi=fig_dpi)
plt.show()

RuntimeError: InSAR and GPS do not share ANY date.

In [38]:
comm_idx

83